In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [15]:
import numpy as np
import pandas as pd

from datetime import datetime
from datetime import timedelta

%matplotlib inline
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense

from sklearn.metrics import mean_squared_error

Read csv's, save as dataframes

In [3]:
oil_df = pd.read_csv("/content/gdrive/MyDrive/grocery_sales/oil.csv")
holiday_df = pd.read_csv("/content/gdrive/MyDrive/grocery_sales/holidays_events.csv")
stores_df = pd.read_csv("/content/gdrive/MyDrive/grocery_sales/stores.csv")
train_df = pd.read_csv("/content/gdrive/MyDrive/grocery_sales/train.csv")

Merge oil and stores df's

In [4]:
train_df = train_df.merge(oil_df, how = 'left', on = 'date')
train_df = train_df.merge(stores_df, how = 'left', on = 'store_nbr')

Initialize extra columns for binary encoding

In [5]:
# initialize holiday location types
train_df['local_holiday'] = 0
train_df['reg_holiday'] = 0
train_df['nat_holiday'] = 0

# initialize store types
train_df['type_a'] = 0
train_df['type_b'] = 0
train_df['type_c'] = 0
train_df['type_d'] = 0
train_df['type_e'] = 0

# initialize holiday types
train_df['holiday_day'] = 0
train_df['addtnl_day'] = 0
train_df['bridge_day'] = 0
train_df['work_day'] = 0
train_df['transfer_day'] = 0

# initialize transferred boolean
train_df['transfer_bool'] = 0

One-hot encode product families

In [6]:
prod_df = pd.get_dummies(train_df.family)
train_df = train_df.join(prod_df)

One-hot encode holidays and store types

In [ ]:
for index, row in holiday_df.iterrows():
    
    if row['transferred'] == True:
        
        train_df['transfer_bool'][train_df['date'] == row['date']] = 1
        
    if row['locale'] == 'National':
        
        train_df['nat_holiday'][train_df['date'] == row['date']] = 1
        
        if row['type'] == 'Holiday':
            train_df['holiday_day'][train_df['date'] == row['date']] = 1  
        elif row['type'] == 'Additional':
            train_df['addtnl_day'][train_df['date'] == row['date']] = 1   
        elif row['type'] == 'Bridge':
            train_df['bridge_day'][train_df['date'] == row['date']] = 1
        elif row['type'] == 'Work':
            train_df['work_day'][train_df['date'] == row['date']] = 1
        elif row['type'] == 'Transfer':
            train_df['transfer_day'][train_df['date'] == row['date']] = 1
        
    elif row['locale'] == 'Regional':
        
        train_df['reg_holiday'][(train_df['date'] == row['date']) 
                                & (train_df['state'] == row['locale_name'])] = 1
        
        if row['type'] == 'Holiday':
            train_df['holiday_day'][(train_df['date'] == row['date']) 
                                    & (train_df['state'] == row['locale_name'])] = 1
        elif row['type'] == 'Additional':
            train_df['addtnl_day'][(train_df['date'] == row['date']) 
                                   & (train_df['state'] == row['locale_name'])] = 1
        elif row['type'] == 'Bridge':
            train_df['bridge_day'][(train_df['date'] == row['date']) 
                                   & (train_df['state'] == row['locale_name'])] = 1
        elif row['type'] == 'Work':
            train_df['work_day'][(train_df['date'] == row['date']) 
                                 & (train_df['state'] == row['locale_name'])] = 1
        elif row['type'] == 'Transfer':
            train_df['transfer_day'][(train_df['date'] == row['date']) 
                                     & (train_df['state'] == row['locale_name'])] = 1
        
    elif row['locale'] == 'Local':
        
        train_df['local_holiday'][(train_df['date'] == row['date']) 
                                  & (train_df['city'] == row['locale_name'])] = 1
        
        if row['type'] == 'Holiday':
            train_df['holiday_day'][(train_df['date'] == row['date']) 
                                    & (train_df['city'] == row['locale_name'])] = 1   
        elif row['type'] == 'Additional':
            train_df['addtnl_day'][(train_df['date'] == row['date']) 
                                   & (train_df['city'] == row['locale_name'])] = 1   
        elif row['type'] == 'Bridge':
            train_df['bridge_day'][(train_df['date'] == row['date']) 
                                   & (train_df['city'] == row['locale_name'])] = 1
        elif row['type'] == 'Work':
            train_df['work_day'][(train_df['date'] == row['date']) 
                                 & (train_df['city'] == row['locale_name'])] = 1   
        elif row['type'] == 'Transfer':
            train_df['transfer_day'][(train_df['date'] == row['date']) 
                                     & (train_df['city'] == row['locale_name'])] = 1

In [ ]:
for index, row in stores_df.iterrows():
    
    if row['type'] == "A":
        train_df['type_a'][train_df['store_nbr'] == row['store_nbr']] = 1
    elif row['type'] == "B":
        train_df['type_b'][train_df['store_nbr'] == row['store_nbr']] = 1  
    elif row['type'] == "C":
        train_df['type_c'][train_df['store_nbr'] == row['store_nbr']] = 1 
    elif row['type'] == "D":
        train_df['type_d'][train_df['store_nbr'] == row['store_nbr']] = 1  
    elif row['type'] == "E":
        train_df['type_e'][train_df['store_nbr'] == row['store_nbr']] = 1

Clean up and view train_df

In [9]:
train_df = train_df.drop(columns=['type', 'city', 'state', 'cluster', 'id', 'family', 'SEAFOOD'])
train_df = train_df.rename(columns={'dcoilwtico': "oil_price"})
train_df['oil_price'] = train_df['oil_price'].fillna(method='bfill')

train_df.head()

,date,store_nbr,sales,onpromotion,oil_price,local_holiday,reg_holiday,nat_holiday,type_a,type_b,...,"LIQUOR,WINE,BEER",MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES
0,2013-01-01,1,0.0,0,93.14,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2013-01-01,1,0.0,0,93.14,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2013-01-01,1,0.0,0,93.14,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2013-01-01,1,0.0,0,93.14,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2013-01-01,1,0.0,0,93.14,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create MLP models for each store

In [11]:
train_df['date'] = pd.to_datetime(train_df['date'])

In [37]:
mlp_train = train_df[train_df['date']<= datetime(2016,5,10)].groupby(['date','store_nbr']).mean().reset_index(['date','store_nbr'])
cols = mlp_train.columns.tolist()

# move sales to the last column
mlp_train = mlp_train[cols[1:2] + cols[3:] + [cols[2]]]

In [35]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [52]:
# choose a number of time steps
n_steps = 7

def fit_mlp(store_nbr):

  # convert into input/output
  X, y = split_sequences(mlp_train[mlp_train["store_nbr"] == store_nbr].to_numpy(), n_steps) 

  # flatten input
  n_input = X.shape[1] * X.shape[2]
  X = X.reshape((X.shape[0], n_input))

  # define model
  model = Sequential()
  model.add(Dense(100, activation='relu', input_dim=n_input))
  model.add(Dense(1))
  model.compile(optimizer='adam', loss='mse')

  # fit model
  model.fit(X, y, epochs=500, verbose=0)

  # demonstrate prediction
  validation = train_df[train_df['date']> datetime(2016,5,10)].groupby(['date','store_nbr']).mean().reset_index(['date','store_nbr'])
  validation = validation[cols[1:2] + cols[3:] + [cols[2]]]

  X_valid, y_valid = split_sequences(validation[validation["store_nbr"] == store_nbr].to_numpy(), n_steps)

  # flatten input
  n_input = X_valid.shape[1] * X_valid.shape[2]
  X_valid = X_valid.reshape((X_valid.shape[0], n_input))

  y_hat = model.predict(X_valid, verbose=0)

  print("The MSE for store " + str(store_nbr) + " is " + str(mean_squared_error(y_hat, y_valid)))

In [53]:
for i in range(1,55):
  fit_mlp(i)

The MSE for store 1 is 17902.100970548607
The MSE for store 2 is 86383.51228120975
The MSE for store 3 is 383386.2466318972
The MSE for store 4 is 24421.88981132976
The MSE for store 5 is 31390.46487252767
The MSE for store 6 is 41372.907311171904
The MSE for store 7 is 149068.7770655555
The MSE for store 8 is 136518.8430766703
The MSE for store 9 is 48286.90650212254
The MSE for store 10 is 3402.632793793283
The MSE for store 11 is 51662.807943928194
The MSE for store 12 is 1809.395829499601
The MSE for store 13 is 2694.686913263808
The MSE for store 14 is 2769.833509973539
The MSE for store 15 is 2755.5297982656543
The MSE for store 16 is 10677.52917685454
The MSE for store 17 is 19815.964652527786
The MSE for store 18 is 18348.40462312023
The MSE for store 19 is 4814.771994912709
The MSE for store 20 is 30290.91653659865
The MSE for store 21 is 10309.056684869212
The MSE for store 22 is 2423.5881838266555
The MSE for store 23 is 2253.661271533557
The MSE for store 24 is 37492.876802